In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
!pip install -q chitra==0.0.20

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

from chitra.trainer import Trainer, create_cnn

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BS = 16

In [ ]:
@tf.function
def rescale(image, label):
    image = tf.cast(image, tf.float32) / 127.5 - 1.0
    return image, label

In [ ]:
@tf.function
def augment(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.random_flip_left_right(image)
    image = image + tf.random.normal((32, 32, 3), mean=0.0, stddev=0.1)
    return image, label

In [ ]:
train_dl = ds_train.map(augment, AUTOTUNE).map(rescale, AUTOTUNE).cache().batch(BS).prefetch(AUTOTUNE)
test_dl = ds_test.map(rescale).batch(BS).prefetch(AUTOTUNE)

In [ ]:
model = create_cnn('mobilenetv2', num_classes=10, drop_out=0.3, weights=None)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10))

In [ ]:
lr_schedule = tfa.optimizers.Triangular2CyclicalLearningRate(1e-3, 1e-2, step_size=5000)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
model.fit(train_dl, epochs=15, validation_data=test_dl)

In [ ]:
model.fit(train_dl, epochs=15, validation_data=test_dl)